In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image as pil_image
sns.set_style('white')
%matplotlib inline

from keras.applications.mobilenet import MobileNet, relu6, DepthwiseConv2D
from keras.preprocessing import image
from keras.layers import AveragePooling2D, Conv2D, UpSampling2D
from keras.models import load_model, Model
from utils.keras_utils import w_accuracy, preprocess_input_tf, center_crop

import itertools
from collections import Counter
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix, cohen_kappa_score


In [ ]:
def plot_confusion_matrix(cm, classes, figname=None, normalize=False, title=None, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    (This function is copied from the scikit docs.)
    """
    plt.figure(figsize=(7,7))
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=18)
    plt.yticks(tick_marks, classes, fontsize=18)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, '%.2f' % cm[i, j], horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",
                     fontsize=16)
        else:
            plt.text(j, i, cm[i, j], horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",
                     fontsize=16)
    plt.tight_layout()
    #plt.ylabel('True label', fontsize=20)
    #plt.xlabel('Predicted label', fontsize=20)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    if title is not None:
        plt.title(title)
    # plt.colorbar()
    if figname is None:
        plt.show()
    else:
        plt.savefig(figname)
        plt.close()

In [ ]:
def plot_stats(y_true, y_pred, n_class=4):
    col = sns.color_palette()

    fig, axarr = plt.subplots(n_class, n_class, sharex=True, figsize=(10,10))
    for i_true in range(n_class):
        v = y_pred[y_true == i_true]
        for j_pred in range(n_class):
            axarr[i_true, j_pred].hist(v[:, j_pred], 10, normed=False, color=col[i_true])
            axarr[i_true, j_pred].set_title('class=%d, prediction=%d' % (i_true, j_pred))
    plt.setp([a.get_xticklabels() for a in axarr[0, :]], visible=False)
    plt.tight_layout()
    plt.show()


In [ ]:
def get_filenames_and_classes(csv_path):
    df = pd.read_csv(csv_path, sep='\t', index_col=0)
    filenames, classes = [], []
    for base_name, primary_grade, sec_grade in zip(df.index, df.iloc[:,0], df.iloc[:,1]):
        primary_grade, sec_grade = int(primary_grade), int(sec_grade)
        filenames.append(base_name)
        classes.append(np.array([primary_grade, sec_grade]).reshape(1,2))
    classes = np.vstack(classes)
    return filenames, classes


In [ ]:
init_dim = 250
target_dim = 224
target_size = (target_dim, target_dim)
input_shape = (target_size[0], target_size[1], 3)
bs = 32

# classes
class_labels = ['Benign', 'Gleason 3', 'Gleason 4', 'Gleason 5']
n_class = len(class_labels)
prefix = '/data3/eirini/dataset_TMA'
patch_dir = os.path.join(prefix, 'train_validation_patches_750')

In [ ]:
# validation set
tma = 'ZT76'
csv_path = os.path.join(prefix, 'tma_info', '%s_gleason_scores.csv' % tma)
val_filenames, val_classes = get_filenames_and_classes(csv_path)
print('Total TMAs in validation set: %d' % len(val_filenames))

In [ ]:
# load model
model_weights = 'model_weights/MobileNet_gleason_weights.h5'
model = load_model(model_weights,
                   custom_objects={'relu6': relu6,
                                   'DepthwiseConv2D': DepthwiseConv2D,
                                   'w_accuracy':w_accuracy
                                       })

outdir = '/data3/eirini/TMA_results/plots'

In [ ]:
# compute confusion matrix for validation set
val_patch_names = []
val_patch_labels = []
for val_subdir in val_filenames:
    subpath = os.path.join(patch_dir, val_subdir)
    for fname in os.listdir(subpath):
        if fname.lower().endswith('.jpg'):
            val_patch_names.append(os.path.join(subpath, fname))
            y = int(fname.split('_')[-1].rstrip('.jpg'))
            val_patch_labels.append(y)
n_patch = len(val_patch_names)

y_pred_prob = np.zeros((n_patch, n_class))
for i, patch_name in enumerate(val_patch_names):
    img = image.load_img(patch_name, grayscale=False, target_size=(init_dim, init_dim))
    X = image.img_to_array(img)
    X = center_crop(X, center_crop_size=target_size)
    X = preprocess_input_tf(X)
    y_pred_prob[i] = model.predict(X[np.newaxis,:,:,:], batch_size=1)[0]

y_true = np.array(val_patch_labels)
y_pred = np.argmax(y_pred_prob, axis=1)

In [ ]:
kappa_val = cohen_kappa_score(y_true, y_pred, weights='quadratic')
print('%.2f' % kappa_val)

In [ ]:
# plot_stats(y_true, y_pred_prob)

In [ ]:

# figname = os.path.join(outdir, 'validation_patch_based.eps')
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, class_labels, figname=None, normalize=True)


In [ ]:
# test filenames
test_patch_dir = os.path.join(prefix, 'inter_observer', 'joint_test_patches_750', 'patho_1')
test_filenames = [f for f in os.listdir(test_patch_dir) if f.startswith('ZT80')]
len(test_filenames)

In [ ]:
patches_csv_path = os.path.join(prefix, 'inter_observer', 'ZT80_patch_grades.csv')
df_patch = pd.read_csv(patches_csv_path, sep='\t', index_col=0)
true_grades = df_patch.values

# make predictions on test cohort patches
y_pred_prob = np.zeros((df_patch.shape[0], n_class))
for i, (patch_name, y_true) in enumerate(zip(df_patch.index, df_patch.iloc[:,0])):
    spot_name = patch_name.split('_patch_')[0]
    full_name = os.path.join(test_patch_dir, spot_name, patch_name+'_class_%d.jpg' % y_true)
    img = image.load_img(full_name, grayscale=False, target_size=(init_dim, init_dim))
    X = image.img_to_array(img)
    X = center_crop(X, center_crop_size=target_size)
    X = preprocess_input_tf(X)
    y_pred_prob[i] = model.predict(X[np.newaxis,:,:,:], batch_size=1)[0]

y_pred = np.argmax(y_pred_prob, axis=1)

In [ ]:
# compute confusion matrix for test set - CNN vs Kim's annotations

# figname = os.path.join(outdir, 'CNN_vs_patho1_patch_based.eps')
cm = confusion_matrix(true_grades[:,0], y_pred)
plot_confusion_matrix(cm, class_labels, figname=None, normalize=True)

kappa_p1 = cohen_kappa_score(true_grades[:,0], y_pred, weights='quadratic')
print('%.2f' % kappa_p1)

In [ ]:
# compute confusion matrix for test set - CNN vs Jan's annotations

# figname = os.path.join(outdir, 'CNN_vs_patho2_patch_based.eps')
cm = confusion_matrix(true_grades[:,1], y_pred)
plot_confusion_matrix(cm, class_labels, figname=None, normalize=True)

kappa_p2 = cohen_kappa_score(true_grades[:,1], y_pred, weights='quadratic')
print('%.2f' % kappa_p2)

In [ ]:
# inter-pathologist variability
cm = confusion_matrix(true_grades[:,0], true_grades[:,1])
# figname = os.path.join(outdir, 'patho1_patho2.eps')
plot_confusion_matrix(cm, class_labels, figname=None, normalize=True)

#cm = confusion_matrix(true_grades[:,1], true_grades[:,0])
#figname = os.path.join(outdir, 'patho2_patho1.eps')
#plot_confusion_matrix(cm, class_labels, figname=None, normalize=True)

kappa_p1_p2 = cohen_kappa_score(true_grades[:,0], true_grades[:,1], weights='quadratic')
print('%.2f' % kappa_p1_p2)

In [ ]:
# Venn diagrams
from matplotlib_venn import venn3
sns.set_palette(sns.color_palette("Set2", 8))
for i in range(n_class):
    set_Kim = set(df_patch.index[true_grades[:,0] == i])
    set_Jan = set(df_patch.index[true_grades[:,1] == i])
    set_CNN = set(df_patch.index[y_pred == i])
    
    plt.figure()
    v = venn3([set_Kim, set_Jan, set_CNN], ['pathologist 1', 'pathologist 2', 'model'])
    for text in v.set_labels:
        text.set_fontsize(18)
    for text in v.subset_labels:
        if text is not None:
            text.set_fontsize(16)
    plt.show()
    #plt.savefig(os.path.join(outdir, 'venn3_class_%d.pdf' % i))
    #plt.close()

In [ ]:
def assign_group(a, b):
    # if both cancer and benign tissue are predicted, ignore benign tissue
    if (a > 0) and (b == 0):
        b = a
    if (b > 0) and (a == 0):
        a = b

    # get the actual Gleason grade
    a += 2
    b += 2
    if a+b <= 6:
        return 1
    elif a+b == 7:
        return 2
    else:
        return 3

def gleason_summary_wsum(y_pred, n_class=4, thres=None):
    gleason_scores = y_pred.copy()
    gleason_scores /= np.sum(gleason_scores)
    # remove outlier predictions
    if thres is not None:
        gleason_scores[gleason_scores < thres] = 0
    # and assign overall grade
    idx = np.argsort(gleason_scores)[::-1]
    primary_class = idx[0]
    secondary_class = idx[1] if gleason_scores[idx[1]] > 0 else idx[0]
    return assign_group(primary_class, secondary_class)

def pil_resize(img, target_size):
    hw_tuple = (target_size[1], target_size[0])
    if img.size != hw_tuple:
        img = img.resize(hw_tuple)
    return img

In [ ]:
# Compute model predictions (pixel-level probability maps) 

w_out, b_out = model.layers[-1].get_weights()
w_out = w_out[np.newaxis,np.newaxis,:,:]

# rescaling factor is 3
big_dim = 1024
base_model = MobileNet(include_top=False, weights=None,
                       input_shape=(big_dim, big_dim, 3),
                       alpha=.5, depth_multiplier=1, dropout=.2)
block_name = 'conv_pw_13_relu'
x_input = base_model.get_layer(block_name).output

# average pooling instead of global pooling
x = AveragePooling2D((7, 7), strides=(1,1), padding='same', name='avg_pool_top')(x_input)
x = Conv2D(n_class, (1, 1), activation='softmax', padding='same')(x)
x_out = UpSampling2D(size=(32, 32), name='upsample')(x)
big_model = Model(base_model.input, x_out)
big_model.load_weights(model_weights, by_name=True)
big_model.layers[-2].set_weights([w_out, b_out])

In [ ]:
# Make predictions on entire TMA spots for the validation cohort.

image_dir = os.path.join(prefix, 'TMA_images')
mask_dir = os.path.join(prefix, 'tissue_masks')
D_val = dict()

for fname in val_filenames:
    full_imfile = os.path.join(image_dir, fname+'.jpg')
    # get network predictions as heatmap
    img = image.load_img(full_imfile, grayscale=False, target_size=(big_dim, big_dim))
    X = image.img_to_array(img)
    X = preprocess_input_tf(X)
    y_pred_prob = big_model.predict(X[np.newaxis,:,:,:], batch_size=1)[0]
    
    # get the (automatically generated) tissue mask
    tissue_maskfile = os.path.join(mask_dir, 'mask_'+fname+'.png')
    tissue_mask = pil_image.open(tissue_maskfile)
    tissue_mask = np.array(pil_resize(tissue_mask, target_size=(big_dim, big_dim)))

    # compute probability only at (predicted) tissue regions
    y_pred_prob[tissue_mask == n_class] = 0.
    y_pred_prob = y_pred_prob.reshape(-1, 4)
    w_sum = np.sum(y_pred_prob, axis=0)
    D_val[fname] = w_sum

In [ ]:
# Compute confusion matrices and Cohen's kappa statistic for Gleason score assignments on entire TMA spots
# (validation cohort)

N = len(D_val.keys())
csv_file = os.path.join(prefix, 'tma_info', 'ZT76_gleason_scores.csv')
df_patho = pd.read_csv(csv_file, sep='\t', index_col=0)

# load training data survival times
ii = 0
x_gleason_cnn = np.zeros(N)
x_gleason_annot_kim = np.zeros(N)
case_ids = []

for fname in D.keys():
    # if the image was annotated by the pathologists
    if fname in df_patho.index:
        a, b = df_patho.loc[fname][['class_primary', 'class_secondary']]
        x_gleason_annot_kim[ii] = assign_group(a, b)
        x_gleason_cnn[ii] = gleason_summary_wsum(D[fname], thres=0.25)
        case_ids.append(fname)
        ii += 1
    
# discard unused part of matrices
set_cnn = x_gleason_cnn[:ii]
set_kim = x_gleason_annot_kim[:ii]
print('Found survival annotation for %d out of %d TMA spots.' % (ii, N))

# compute kappa values
kappa_p1 = cohen_kappa_score(set_kim, set_cnn, weights='quadratic')
print('CNN-pathologist 1: %.2f' % kappa_p1)

classes = ['benign', 'Gleason 6', 'Gleason 7', 'Gleason 8', 'Gleason 9', 'Gleason 10']
# figname = os.path.join(outdir, 'validation_TMA_based.eps')
cm = confusion_matrix(set_kim, set_cnn)
plot_confusion_matrix(cm, classes, figname=None, normalize=False)


In [ ]:
# Make predictions on entire TMA spots for the test cohort.

image_dir = os.path.join(prefix, 'TMA_images')
mask_dir = os.path.join(prefix, 'tissue_masks')
D = dict()

for fname in test_filenames:
    full_imfile = os.path.join(image_dir, fname+'.jpg')
    # get network predictions as heatmap
    img = image.load_img(full_imfile, grayscale=False, target_size=(big_dim, big_dim))
    X = image.img_to_array(img)
    X = preprocess_input_tf(X)
    y_pred_prob = big_model.predict(X[np.newaxis,:,:,:], batch_size=1)[0]
    
    # get the (automatically generated) tissue mask
    tissue_maskfile = os.path.join(mask_dir, 'mask_'+fname+'.png')
    tissue_mask = pil_image.open(tissue_maskfile)
    tissue_mask = np.array(pil_resize(tissue_mask, target_size=(big_dim, big_dim)))

    # compute probability only at (predicted) tissue regions
    y_pred_prob[tissue_mask == n_class] = 0.
    y_pred_prob = y_pred_prob.reshape(-1, 4)
    w_sum = np.sum(y_pred_prob, axis=0)
    D[fname] = w_sum

In [ ]:
# Compute confusion matrices and Cohen's kappa statistic for Gleason score assignments on entire TMA spots
# (test cohort)

N = len(D.keys())
csv_file = os.path.join(prefix, 'tma_info', 'ZT80_gleason_scores.csv')
df_patho = pd.read_csv(csv_file, sep='\t', index_col=0)

ii = 0
x_gleason_cnn = np.zeros(N)
x_gleason_annot_kim = np.zeros(N)
x_gleason_annot_jan = np.zeros(N)
case_ids = []

for fname in D.keys():
    # if the image was annotated by the pathologists
    if fname in df_patho.index:
        a, b = df_patho.loc[fname][['kim_class_primary', 'kim_class_secondary']]
        x_gleason_annot_kim[ii] = assign_group(a, b)
        a, b = df_patho.loc[fname][['jan_class_primary', 'jan_class_secondary']]
        x_gleason_annot_jan[ii] = assign_group(a, b)
        x_gleason_cnn[ii] = gleason_summary_wsum(D[fname], thres=0.25)
        case_ids.append(fname)
        ii += 1
    
# discard unused part of matrices
set_cnn = x_gleason_cnn[:ii]
set_kim = x_gleason_annot_kim[:ii]
set_jan = x_gleason_annot_jan[:ii]
print('Found survival annotation for %d out of %d TMA spots.' % (ii, N))

# compute kappa values
kappa_p1_p2 = cohen_kappa_score(set_kim, set_jan, weights='quadratic')
print('inter-pathologist: %.2f' % kappa_p1_p2)
kappa_p1 = cohen_kappa_score(set_kim, set_cnn, weights='quadratic')
print('CNN-pathologist 1: %.2f' % kappa_p1)
kappa_p2 = cohen_kappa_score(set_jan, set_cnn, weights='quadratic')
print('CNN-pathologist 2: %.2f' % kappa_p2)

# confusion matrices
classes = ['benign', 'Gleason 6', 'Gleason 7', 'Gleason 8', 'Gleason 9', 'Gleason 10']

#figname = os.path.join(outdir, 'CNN_vs_patho1_TMA_based.eps')
cm = confusion_matrix(set_kim, set_cnn)
plot_confusion_matrix(cm, classes, figname=None, normalize=False)

#figname = os.path.join(outdir, 'CNN_vs_patho2_TMA_based.eps')
cm = confusion_matrix(set_jan, set_cnn)
plot_confusion_matrix(cm, classes, figname=None, normalize=False)

#figname = os.path.join(outdir, 'patho1_vs_patho2_TMA_based.eps')
cm = confusion_matrix(set_kim, set_jan)
plot_confusion_matrix(cm, classes, figname=None, normalize=False)

In [ ]:
# Read in survival data

df_surv = pd.read_csv(os.path.join(prefix, 'tma_info', 'filtered_survival_and_clinical_data.csv'),
                      index_col=0)
df_surv = df_surv.replace(to_replace=' ', value=np.nan)
surv_pairs = [('os', 'st_os_gen'), ('os', 'st_os_spec'), ('rfs', 'rfs_status')]

In [ ]:
# Organize Gleason score group assignments and survival data in compact dataframes
# for downstream analysis in R

N = len(D.keys())
csv_file = os.path.join(prefix, 'tma_info', 'ZT80_gleason_scores.csv')
df_patho = pd.read_csv(csv_file, sep='\t', index_col=0)

for surv_time, surv_status in surv_pairs:
    print(surv_status)

    # load training data survival times
    ii = 0
    y_surv = np.zeros((N, 2))
    x_gleason_cnn = np.zeros((N, 1))
    x_gleason_annot_kim = np.zeros((N, 1))
    x_gleason_annot_jan = np.zeros((N, 1))
    case_ids = []

    for fname in D.keys():
        # if survival information exists and image was annotated by both pathologists
        if (fname in df_surv.index) and (fname in df_patho.index):
            y_surv[ii] = df_surv.loc[fname][[surv_time, surv_status]]
            a, b = df_patho.loc[fname][['kim_class_primary', 'kim_class_secondary']]
            x_gleason_annot_kim[ii, 0] = assign_group(a, b)
            a, b = df_patho.loc[fname][['jan_class_primary', 'jan_class_secondary']]
            x_gleason_annot_jan[ii, 0] = assign_group(a, b)
            x_gleason_cnn[ii, 0] = gleason_summary_wsum(D[fname], thres=0.25)
            case_ids.append(fname)
            ii += 1
            
    # discard unused part of matrices
    y_surv = y_surv[:ii]
    x_gleason_cnn = x_gleason_cnn[:ii]
    x_gleason_annot_kim = x_gleason_annot_kim[:ii]
    x_gleason_annot_jan = x_gleason_annot_jan[:ii]
    print('Found survival information for %d out of %d TMA spots.' % (ii, N))
    print('Events occurred: %d' % np.nansum(y_surv[:,1] == 1))
    
    # write out the dataframe
    df = pd.DataFrame(np.hstack([y_surv, x_gleason_cnn, x_gleason_annot_kim, x_gleason_annot_jan]),
                      columns=['time', 'status'] + ['GL_predicted', 'GL_annot_kim', 'GL_annot_jan'],
                      index=case_ids)
    df = df.dropna()
    df.to_csv(os.path.join(outdir, '%s.csv' % surv_status), index=True, index_label='case_id')
    


In [ ]:
# Venn diagrams for Gleason score group assignments

from matplotlib_venn import venn3
risk_groups = ['low', 'intermediate', 'high']
for _surv_time, surv_status in surv_pairs:
    df = pd.read_csv(os.path.join(outdir, '%s.csv' % surv_status))
    for i in range(1, 4):
        print('%s-risk group' % risk_groups[i-1])
        set_Kim = set(df.index[df['GL_annot_kim'] == i])
        set_Jan = set(df.index[df['GL_annot_jan'] == i])
        set_CNN = set(df.index[df['GL_predicted'] == i])

        plt.figure()
        v = venn3([set_Kim, set_Jan, set_CNN], ['pathologist 1', 'pathologist 2', 'model'])
        for text in v.set_labels:
            text.set_fontsize(18)
        for text in v.subset_labels:
            if text is not None:
                text.set_fontsize(16)
        plt.show()
        #plt.savefig(os.path.join(outdir, 'venn3_%s_Gleason_group_%d.pdf' % (surv_status, i)))
        #plt.close()

In [ ]:
# compute kappa values for Gleason score group assignment agreements
for _surv_time, surv_status in surv_pairs:
    df = pd.read_csv(os.path.join(outdir, '%s.csv' % surv_status))
    set_Kim = np.array(df['GL_annot_kim'])
    set_Jan = np.array(df['GL_annot_jan'])
    set_CNN = np.array(df['GL_predicted'])
    
    kappa_p1_p2 = cohen_kappa_score(set_Kim, set_Jan, weights='quadratic')
    print('inter-pathologist: %.2f' % kappa_p1_p2)
    kappa_p1 = cohen_kappa_score(set_Kim, set_CNN, weights='quadratic')
    print('CNN-pathologist 1: %.2f' % kappa_p1)
    kappa_p2 = cohen_kappa_score(set_Jan, set_CNN, weights='quadratic')
    print('CNN-pathologist 2: %.2f' % kappa_p2)
    print('\n')